In [ ]:
import requests
import string
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from unicodedata import numeric
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
import random
#https://stackoverflow.com/questions/28639677/capitalize-the-first-letter-after-a-punctuation/28639714

In [ ]:
URL = "https://www.allrecipes.com/recipe/11679/homemade-mac-and-cheese/"

In [ ]:
class Ingredient:

  def __init__(self,fullname=None,main_ingredients=None,quantity_type=None,quantity = None,units=None,descriptor=None,preparation=None,ingredient_type=None):  
    self.fullname = fullname
    self.main_ingredients = main_ingredients
    self.quantity_type = quantity_type
    self.quantity = quantity
    self.units = units
    self.descriptor = descriptor
    self.preparation = preparation
    self.ingredient_type = ingredient_type

  def print_ingredient(self):
    print('Full Name: ',self.fullname)
    print("Main Ingredient: ",self.main_ingredients)
    print("Quantity Type: ",self.quantity_type)
    print("Quantity: ",self.quantity)
    print("Units: ",self.units)
    print('Descriptor: ',self.descriptor)
    print('Preparation: ',self.preparation,'\n')

In [ ]:
class Step:
  def __init__(self,full_step=None,ingredients=None,tools=None,methods = None,times=None):  
    self.full_step = full_step
    self.ingredients = ingredients
    self.tools = tools
    self.methods = methods
    self.times = times
  
  def print_steps(self):
    print('Full Step: ',self.full_step)
    print("Ingredients: ",self.ingredients)
    print("Tools: ",self.tools)
    print("Methods: ",self.methods)
    print("Times: ",self.times,'\n')

In [ ]:
def get_ingredients_tools_methods_steps(URL):
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  ingredient_parse = soup.find_all("input", class_="checkbox-list-input")
  direction_parse = soup.find_all("div", class_="paragraph")
  unicode_quantities = []
  unicode_units = []
  unicode_ingredients = []
  unicode_type = []

  #----------------------------Parsing Ingredients--------------------------#
  for ingredient in ingredient_parse:
    val = ingredient.attrs
    if u'data-quantity' in val.keys():
        unicode_quantities.append(val[u'data-init-quantity'])
    if u'data-unit' in val.keys():
        unicode_units.append(val[u'data-unit'])
    if u'data-ingredient' in val.keys():
        unicode_ingredients.append(val[u'data-ingredient'])
    if u'data-ingredient' in val.keys():
        unicode_type.append(val[u'data-unit_family'])

  quantities = unicode_quantities
  units = unicode_units
  ingredients = unicode_ingredients
  quantity_type = unicode_type
  descriptor  = [None] * len(ingredients)
  preparation = [None] * len(ingredients)
  main_ingredients = [''] * len(ingredients)

  def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent
  
  for i in range(len(ingredients)):
    tokens = preprocess(ingredients[i])
    for token in tokens:
      if token[1] == 'JJ':
        descriptor[i] = token[0]
      elif token[1] in ['VBD','VBN']:
        preparation[i] = token[0]
      elif token[1] in ['NN','NNS','NNP','NNPS']:
        main_ingredients[i] += token[0]+' '
    main_ingredients[i] = main_ingredients[i].rstrip()

  Ingredients = [0] * len(ingredients)
  for i in range(len(ingredients)):
    new_ingredient = Ingredient(fullname=ingredients[i],main_ingredients=main_ingredients[i],quantity_type=quantity_type[i],quantity = quantities[i],units=units[i],descriptor=descriptor[i],preparation=preparation[i],ingredient_type=None)
    Ingredients[i] = new_ingredient
  
  #----------------------------Ingredients are ready--------------------------#


  #----------------------------Half Parsing Directions--------------------------#
  directions = []

  for step in direction_parse:
    step_text = step.text
    step_text.encode("ascii", "ignore").strip()
    directions.append(step_text)

  #--------------------------Half Parsed Directions are ready------------#

  #----------------------------Parsing Tools--------------------------#
  with open("/content/drive/MyDrive/NLP2/list_of_tools.txt") as file:
      lines = file.readlines()
      list_of_tools = [line.rstrip() for line in lines]

  Tools = []
  for i in range(len(directions)):
    direction_lower = directions[i]
    for tool in list_of_tools:
      if tool in direction_lower:
        Tools.append(tool)

  #----------------------------Tools are ready--------------------------#

  #----------------------------Parsing Methods--------------------------#
  Cooking_Methods = []
  with open("/content/drive/MyDrive/NLP2/list_of_cooking_methods.txt") as file:
      lines = file.readlines()
      list_of_cooking_methods = [line.rstrip() for line in lines]
  
  lemmatizer = WordNetLemmatizer()
  for i in range(len(directions)):
    direction_lower = directions[i].lower()
    for cooking_method in list_of_cooking_methods:
      if cooking_method in direction_lower or lemmatizer.lemmatize(cooking_method) in direction_lower:
        Cooking_Methods.append(cooking_method)
        
  
  #----------------------------Methods are ready--------------------------#

  #----------------------------Parsing Directions--------------------------#
  Steps = []

  for i in range(len(directions)):
    direction_lower = directions[i].lower()
    step_wise_ingredients = []
    step_wise_tools = []
    step_wise_methods = []
    step_wise_times = ''
    
    for tool in Tools:
      if tool in direction_lower and tool not in step_wise_tools:
        step_wise_tools.append(tool)
    
    for method in Cooking_Methods:
      if method in direction_lower and method not in step_wise_methods:
        step_wise_methods.append(method)
    
    for ingredient in Ingredients:
      if ingredient.main_ingredients in direction_lower and ingredient.fullname not in step_wise_ingredients :
        step_wise_ingredients.append(ingredient.fullname)
    
    time_keywords = 'minutes|seconds|hours|secs|hrs|mins|min'
    if (re.search(time_keywords,direction_lower)):
      index = re.search(time_keywords,direction_lower).start()
      new_str = direction_lower[:index]
      step_wise_times+= str(new_str.split()[-1])
      unit = re.findall(time_keywords,direction_lower)[-1]
      step_wise_times += ' '+str(unit)
    new_step = Step (full_step=directions[i],ingredients=step_wise_ingredients,tools=step_wise_tools,methods = step_wise_methods,times=step_wise_times)
    Steps.append(new_step)
    
  return Ingredients, Tools, Cooking_Methods, Steps
 #----------------------------Directions are ready--------------------------#



In [ ]:
#Test intial data collection
Ingredients, Tools, Cooking_Methods, Steps = get_ingredients_tools_methods_steps(URL)

In [ ]:
for ingredient in Ingredients:
  ingredient.print_ingredient()

In [ ]:
for tool in Tools:
  print(tool)

In [ ]:
for method in Cooking_Methods:
  print(method)

In [ ]:
for step in Steps:
  step.print_steps()

In [ ]:
#This code coverts any '5 pounds of diced lean chicken into a parsed form'
#Needed for when we are substituing an old ingredient with a new ingredient
def transform_ingredient(full_ingredient):
  def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

  full_ingredient = full_ingredient.lower()
  transformed_ingredient = Ingredient()
  transformed_ingredient.fullname = full_ingredient
  
  with open("/content/drive/MyDrive/NLP2/list_of_measurement_units.txt") as file:
      lines = file.readlines()
      list_of_measurement_units = [line.rstrip() for line in lines]

  tokens = preprocess(full_ingredient)
  quantity_type = ''
  quantity = ''
  descriptor  = ''
  preparation = ''
  main_ingredients = ''
  units = ''
      
  for token in tokens:
    if token[0] in list_of_measurement_units:
      units = token[0]
    elif token[1] == 'CD':
      quantity = token[0]    
    elif token[1] == 'JJ':
      descriptor = token[0]
    elif token[1] in ['VBD','VBN']:
      preparation = token[0]
    elif token[1] in ['NN','NNS','NNP','NNPS']:
      main_ingredients += token[0]+' '
  main_ingredients = main_ingredients.rstrip()
  
  transformed_ingredient.main_ingredients = main_ingredients

  if units in ['cups','cup','tablespoons','tablespoon','teaspoons','teaspoon','gallons','gallon']:
    transformed_ingredient.quantity_type = 'volumetric'
  elif units in ['pounds']:
    transformed_ingredient.quantity_type = 'weight'
  elif units in ['pinch']:
    transformed_ingredient.quantity_type = 'pinch'
  
  transformed_ingredient.quantity = quantity
  transformed_ingredient.units = units
  transformed_ingredient.descriptor = descriptor
  transformed_ingredient.preparation = preparation
  return transformed_ingredient

In [ ]:
#Substitutes any ingredient everywhere in the recipe and the ingredient
#old_ingredient should just be 'beef' , 'lemon' or something like that
#new_ingredient should be an Ingredient object
#transform_ingredient will transform any ingredient like '5 pounds lean chicken' into this form
def substitute_ingredient_fn(Ingredients, Tools, Cooking_Methods, Steps,old_ingredient, substitute_ingredient):
  new_Ingredients = [0] * len(Ingredients)
  new_Tools = [0] * len (Tools)
  new_Cooking_Methods = [0] * len(Cooking_Methods)
  new_Steps = [0] * len(Steps)
  
  for i in range(len(Ingredients)):
    if old_ingredient.lower() in Ingredients[i].fullname or old_ingredient.lower() in Ingredients[i].main_ingredients:
      new_Ingredients[i] = substitute_ingredient
    else:
      new_Ingredients[i] =  Ingredients[i]

  new_Tools = Tools
  new_Cooking_Methods = Cooking_Methods
  
  for i in range(len(Steps)):
    if old_ingredient.lower() not in Steps[i].full_step.lower() and old_ingredient.lower() not in Steps[i].full_step.lower():
      new_Steps[i] = Steps[i]
    
    else:
      temporary_step = Step(full_step=Steps[i].full_step,ingredients=Steps[i].ingredients,tools=Steps[i].tools,methods = Steps[i].methods,times=Steps[i].times)
      for j in range(len(temporary_step.ingredients)):
        if old_ingredient.lower() in temporary_step.ingredients[j]:
          temporary_step.ingredients[j] = substitute_ingredient.fullname
      

      if old_ingredient.lower() in temporary_step.full_step.lower():
        temporary_step.full_step = temporary_step.full_step.lower().replace(old_ingredient.lower(),substitute_ingredient.main_ingredients)
        temporary_step.full_step = '. '.join(map(lambda s: s.strip().capitalize(), temporary_step.full_step.split('.')))
        new_Steps[i] = temporary_step



  return new_Ingredients, new_Tools, new_Cooking_Methods, new_Steps

In [1]:
#An example substituting milk with spahgetti squash
Ingredients, Tools, Cooking_Methods, Steps = get_ingredients_tools_methods_steps(URL)
old_ingredient = 'milk'
substitute_ingredient = transform_ingredient('5 Pounds Large Seeded Spaghetti Squash')
Sub_Ingredients, Sub_Tools, Sub_Cooking_Methods, Sub_Steps = substitute_ingredient_fn(Ingredients, Tools, Cooking_Methods, Steps,old_ingredient, substitute_ingredient)

NameError: ignored

In [ ]:
for ingredient in Sub_Ingredients:
  ingredient.print_ingredient()

In [ ]:
for step in Sub_Steps:
  step.print_steps()

In [ ]:
#uses the subsitute ingredient helper function to transform the recipie into veg. randomly selects a veg alternative
def to_veg(Ingredients, Tools, Cooking_Methods, Steps):
  with open("/content/drive/MyDrive/NLP2/list_of_non_veg_items.txt") as file:
    lines = file.readlines()
    list_of_non_veg_items = [line.rstrip() for line in lines]

  with open("/content/drive/MyDrive/NLP2/list_of_veg_substitutes.txt") as file:
    lines = file.readlines()
    list_of_veg_substitutes = [line.rstrip() for line in lines]
  
  old_ingredient = None
  for i in range(len(Ingredients)):
    for non_veg_item in list_of_non_veg_items:
      if non_veg_item in Ingredients[i].fullname.lower() or non_veg_item in Ingredients[i].main_ingredients.lower():
       old_ingredient = non_veg_item

  substitute_ingredient = transform_ingredient(random.choice(list_of_veg_substitutes))
  if (old_ingredient):
    return substitute_ingredient_fn(Ingredients, Tools, Cooking_Methods, Steps,old_ingredient, substitute_ingredient)
  else:
    return Ingredients, Tools, Cooking_Methods, Steps

In [ ]:
#testing to veg function
URL_nonveg = 'https://www.allrecipes.com/recipe/258947/mushroom-beef-burgers/'
Ingredients, Tools, Cooking_Methods, Steps = get_ingredients_tools_methods_steps(URL_nonveg)
Sub_Ingredients, Sub_Tools, Sub_Cooking_Methods, Sub_Steps = to_veg(Ingredients, Tools, Cooking_Methods, Steps)

In [ ]:
for ingredient in Sub_Ingredients:
  ingredient.print_ingredient()

In [ ]:
for step in Sub_Steps:
  step.print_steps()

In [ ]:
def ratio(Ingredients, Tools, Cooking_Methods, Steps,ratio_num):
  for i in range(len(Ingredients)):
    if Ingredients[i].quantity.isnumeric():
      Ingredients[i].quantity=float(Ingredients[i].quantity)*ratio_num
  else:
    return Ingredients, Tools, Cooking_Methods, Steps

In [ ]:
URL_nonveg = 'https://www.allrecipes.com/recipe/258947/mushroom-beef-burgers/'
Ingredients, Tools, Cooking_Methods, Steps = get_ingredients_tools_methods_steps(URL_nonveg)

Sub_Ingredients, Sub_Tools, Sub_Cooking_Methods, Sub_Steps = ratio(Ingredients, Tools, Cooking_Methods, Steps,2)